In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys
import pymedphys._mocks.profiles
import pymedphys._gamma.implementation.shell
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findbb
import pymedphys._wlutz.core
import pymedphys._wlutz.reporting

In [ ]:
image_path = pymedphys.data_path('wlutz_image.png')
image_path

In [ ]:
x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)
field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)

edge_lengths = [20, 20]
bb_diameter = 8
bb_radius = bb_diameter / 2
penumbra = 2

bb_centre, field_centre, field_rotation = pymedphys._wlutz.core.find_field_and_bb(
    x, y, img, edge_lengths, bb_diameter, penumbra=penumbra
)

bb_centre, field_centre, field_rotation

In [ ]:
%timeit pymedphys._wlutz.core.find_field_and_bb(x, y, img, edge_lengths, bb_diameter, penumbra=penumbra)

In [ ]:
def create_bb_to_minimise(field, bb_diameter):

    points_to_check, dist = pymedphys._wlutz.interppoints.create_bb_points_function(bb_diameter)
    dist_mask = np.unique(dist)[:, None] == dist[None, :]

    def to_minimise(centre):
        x, y = points_to_check(centre)

        total_minimisation = 0

        for current_mask in dist_mask[1::]:
            current_layer = field(x[current_mask], y[current_mask])
            total_minimisation += np.mean((current_layer - np.mean(current_layer)) ** 2)

        return total_minimisation / (len(dist_mask) - 1)

    return to_minimise

In [ ]:
to_minimise = create_bb_to_minimise(field, bb_diameter)

In [ ]:
%timeit to_minimise(bb_centre)

In [ ]:
reference_val = to_minimise(bb_centre)
reference_val

In [ ]:
points_to_check, dist = pymedphys._wlutz.interppoints.create_bb_points_function(bb_diameter)
dist_mask = np.unique(dist)[:, None] == dist[None, :]

In [ ]:
x, y = points_to_check(bb_centre)
results = field(x, y)
results

In [ ]:
num_in_mask = np.sum(dist_mask, axis=1)
mask_count_per_item = np.sum(num_in_mask[:,None] * dist_mask, axis=0)
mask_count_per_item

In [ ]:
masked_results = results * dist_mask
masked_results

In [ ]:
mask_mean = np.sum(masked_results, axis=1) / num_in_mask
mask_mean

In [ ]:
mask_mean_lookup = np.where(dist_mask)[0]
mask_mean_lookup

In [ ]:
diff_to_mean_square = (results - mask_mean[mask_mean_lookup])**2
np.sum(diff_to_mean_square / mask_count_per_item) / (len(mask_mean) - 1)

In [ ]:
def create_bb_to_minimise_fast(field, bb_diameter):

    points_to_check, dist = pymedphys._wlutz.interppoints.create_bb_points_function(bb_diameter)
    dist_mask = np.unique(dist)[:, None] == dist[None, :]
    num_in_mask = np.sum(dist_mask, axis=1)
    mask_count_per_item = np.sum(num_in_mask[:,None] * dist_mask, axis=0)
    mask_mean_lookup = np.where(dist_mask)[0]

    def to_minimise(centre):
        x, y = points_to_check(centre)
        
        results = field(x, y)
        masked_results = results * dist_mask
        mask_mean = np.sum(masked_results, axis=1) / num_in_mask
        diff_to_mean_square = (results - mask_mean[mask_mean_lookup])**2
        mean_of_layers = np.sum(diff_to_mean_square[1::] / mask_count_per_item[1::]) / (len(mask_mean) - 1)
        
        return mean_of_layers

    return to_minimise

In [ ]:
to_minimise_fast = create_bb_to_minimise_fast(field, bb_diameter)

In [ ]:
%timeit to_minimise(bb_centre)

In [ ]:
%timeit to_minimise_fast(bb_centre)

In [ ]:
pymedphys._wlutz.reporting.image_analysis_figure(
    x,
    y,
    img,
    bb_centre,
    field_centre,
    field_rotation,
    bb_diameter,
    edge_lengths,
    penumbra,
)